In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
# helper functions
def haversine(lat1, lon1, lat2, lon2): 
      
    # distance between latitudes 
    # and longitudes 
    dLat = (lat2 - lat1) * np.pi / 180.0
    dLon = (lon2 - lon1) * np.pi / 180.0
  
    # convert to radians 
    lat1 = (lat1) * np.pi / 180.0
    lat2 = (lat2) * np.pi / 180.0
  
    # apply formulae 
    a = (pow(np.sin(dLat / 2.), 2) + 
         pow(np.sin(dLon / 2.), 2) * 
             np.cos(lat1) * np.cos(lat2)); 
    rad = 6371.
    c = 2. * np.arcsin(np.sqrt(a)) 
    return rad * c 

In [3]:
df_google = pd.read_csv('dataFrameAll_.csv')
user_opt = {'Food': True,'Culture': 0.25,'Nature':0.15,'Sight Seeing':0.1,'Museum':0.2,'Shopping':0.3}
n_before = 5
n_after = 2

In [4]:
class route:
    
    part1 = []
    part2 = []
    
    food = []
    
    def get_route_no_dist(self):
        return pd.concat([self.part1,self.food,self.part2])
    
    def compute_distances(self,startpoint):
        r = self.get_route_no_dist()
        
        x = r.lng.values
        y = r.lat.values
        distances = []
        for i in range(len(r)):
            if i==0:
                #dist = np.sqrt( (x[i]-startpoint[0])**2+(y[i]-startpoint[1])**2 )
                dist = haversine(y[i],x[i],startpoint[1],startpoint[0])
            else:
                #dist = np.sqrt( (x[i]-x[i-1])**2+(y[i]-y[i-1])**2 )
                dist = haversine(y[i],x[i],y[i-1],x[i-1])
            distances.append(dist)
        return distances
    
    def get_route(self,startpoint):
        r = self.get_route_no_dist()
        distances = self.compute_distances(startpoint)
        r['distances'] = distances
        return r
    def route_score(self, startpoint, class_weights):
        score = 0
        r = self.get_route(startpoint)
        
        for i in range(len(r)):
            
            c_weight = float(class_weights[r.iloc[i].type])
            path = r.iloc[i].distances
            if path == 0:
                path = 0.0001
            score_i = ((r.iloc[i].rating)/path)*c_weight
            #score_i = ((r.iloc[i].rating)/(r.iloc[i].distances))*c_weight
            score+=score_i
            
        return score
        
def generate_route(locations,add_food,food_locations):
    
    r = route()
    part1_len = n_before
    part2_len = n_after
    #part1_len = np.random.randint(1,4)
    #part2_len = np.random.randint(1,4)
    
    track = np.random.choice(range(len(locations)),part1_len+part2_len,replace=False)
    
    r.part1 = locations.iloc[ track[:part1_len] ]
    r.part2 = locations.iloc[ track[part1_len:] ]
    
    if add_food:
        food_idx = np.random.choice(range(len(food_locations)))
        r.food = food_locations.iloc[[food_idx]]
    
    return r

def generate_routes(n, locations,add_food,food_locations):
    
    routes = []
    for i in range(n):
        routes.append( generate_route(locations,add_food,food_locations) )
    return routes

def pick_1outof2(s1,s2):
    
    rand_n = np.random.rand()
    
    thresh = s1/(s1+s2)
    
    if rand_n > thresh:
        return 1
    else:
        return 0

def create_child(r1,score1,r2,score2):
    
    child = route()
    
    parents = [r1,r2]
    
    is_food = len(r1.food)
    if is_food:
        which_food = pick_1outof2(score1,score2)
        child.food = parents[which_food].food
    
    
    part1_parent = pick_1outof2(score1,score2)
    part2_parent = pick_1outof2(score1,score2)
    
    child.part1 = parents[part1_parent].part1
    child.part2 = parents[part2_parent].part2
    
    return child
    
def check_child(child):
    start = [-1,1]
    r = child.get_route(start)
    if 0.0 in r.distances.values:
        return False
    else:
        return True
    
def mate_parents(r_list,startpoint,class_weights):
    
    children = []
    
    alpha = r_list[0]
    score1 = alpha.route_score(startpoint,class_weights)
    
    for par_i in range(1,len(r_list)):
        beta = r_list[par_i]
        score2 = beta.route_score(startpoint,class_weights)
        
        child = create_child(alpha,score1,beta,score2)
        if check_child(child):
            children.append(child)
        else:
            child = create_child(alpha,score1,beta,score2)
            if check_child(child):
                children.append(child)
    return children

In [5]:
def plot_route(start,r,ax):
    
    x = r.get_route(start).lng
    y = r.get_route(start).lat
    x = [start[0]]+list(x)
    y = [start[1]]+list(y)
    ax.plot(x,y)
    
def get_route_coordinates(start,r,ax):
    lat_coord = []
    lng_coord = []
    x = r.get_route(start).lng
    y = r.get_route(start).lat
    x = [start[0]]+list(x)
    y = [start[1]]+list(y)
    for i in range(len(x)):
        lng_coord.append(x[i])
        lat_coord.append(y[i])
    return lng_coord, lat_coord
def get_best_route(startingpoint, loc_df, food_df, do_food, class_weights):
    n_in_gen = 15
    n_top = 5
    n_generations = 50
    
    generation_record = []
    
    current_gen = generate_routes(n_in_gen, loc_df, do_food ,food_df)
    
    for gen_i in range(n_generations):
        scores = [ x.route_score(startingpoint, class_weights) for x in current_gen ]
        topscores = sorted(zip(current_gen,scores),key=lambda pair: pair[1])
        current_gen = [x for x,_ in topscores]
        generation_record.append(current_gen)
        
        best_ones = current_gen[-n_top:]
        
        children = mate_parents(best_ones,startingpoint,class_weights)
        n_children = len(children)
        
        new_blood = []
        if n_in_gen-n_top-n_children > 0:
            new_blood = generate_routes(n_in_gen-n_top-n_children, loc_df, do_food ,food_df)
        current_gen = best_ones+children+new_blood
        
        
        
        
    scores = [x.route_score(startingpoint, class_weights) for x in current_gen]
    topscores = sorted(zip(current_gen,scores),key=lambda pair: pair[1])[-1]
    best_one = topscores
    
    return best_one,generation_record

In [6]:
def getPlace(dataFrame,starting_point,user_opt):
    
    loc_df = dataFrame[dataFrame.type!='Food']
    food_df = dataFrame[dataFrame.type=='Food']
    
    best_one,generation_record = get_best_route(startingpoint=starting_point, loc_df=loc_df, food_df=food_df, do_food=True, class_weights=user_opt)
    fig,ax = plt.subplots(figsize=(8,8))
    start = starting_point.copy()
    
    ax.scatter(loc_df.lng,loc_df.lat)
    ax.scatter(food_df.lng,food_df.lat,c='r')
    plt.close()
    r_list = generate_routes(100,loc_df,True,food_df)

    #for r in r_list:
    #    plot_route(start,r,ax)
    #ax.scatter(start[0],start[1],s=130,c='g')
    
    lng_c, lat_c = get_route_coordinates(start,r_list[-1],ax)
    #plt.show()
    names = []
    types = []
    distance = []
    for i in range(len(lng_c)-1):
        names.append(list(dataFrame[(dataFrame['lng'] == lng_c[i+1]) & (dataFrame['lat'] == lat_c[i+1])]['name'].values))
        types.append(list(dataFrame[(dataFrame['lng'] == lng_c[i+1]) & (dataFrame['lat'] == lat_c[i+1])]['type'].values))
        if i == 0:
            distance.append(haversine(starting_point[1],starting_point[0],lat_c[i+1],lng_c[i+1]))
        else:
            distance.append(haversine(lat_c[i],lng_c[i],lat_c[i+1],lng_c[i+1]))
        
    return names,types, distance

In [7]:
places, types_of_attraction,distance = getPlace(dataFrame=df_google,starting_point=[34.8,32.1],user_opt=user_opt)

In [8]:
places

[['MennoTours'],
 ['Ilana Goor Museum'],
 ['Rishon LeTsiyon Beach'],
 ['Museum of the Jewish People at Beit Hatfutsot'],
 ['TAL Aviation Israel'],
 ['Mashya'],
 ['HaSela Beach'],
 ['Herzlilienblum Museum']]

In [9]:
types_of_attraction

[['Sight Seeing'],
 ['Museum'],
 ['Nature'],
 ['Museum'],
 ['Sight Seeing'],
 ['Food'],
 ['Nature'],
 ['Museum']]

In [ ]:
distance